In [60]:
import pandas as pd
from datetime import datetime

In [66]:
players = pd.read_csv("data/players.csv")

In [29]:
players.dtypes

nflId                int64
height              object
weight               int64
birthDate           object
collegeName         object
officialPosition    object
displayName         object
dtype: object

In [68]:
today = datetime.today()
players['birthDate'] = pd.to_datetime(players['birthDate'])

players['age'] = players['birthDate'].apply(
               lambda x: today.year - x.year - 
               ((today.month, today.day) < (x.month, x.day)) 
               )

In [69]:
players.head()

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName,age
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,45.0
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger,40.0
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,40.0
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,39.0
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick,40.0


In [24]:
players[["officialPosition", "weight"]].groupby("officialPosition").mean().sort_values("weight")

,weight
officialPosition,
CB,193.109375
DB,198.000000
WR,199.366667
FS,202.361111
SS,204.231884
RB,214.190476
QB,220.733333
MLB,233.259259
ILB,235.567901


In [71]:
players[["officialPosition", "age"]].groupby("officialPosition").describe()

age                                                    
                  count       mean       std   min   25%   50%    75%   max
officialPosition                                                           
C                  49.0  28.673469  2.988783  24.0  26.0  29.0  31.00  37.0
CB                158.0  27.512658  2.708179  23.0  26.0  27.0  29.00  35.0
DB                  1.0  24.000000       NaN  24.0  24.0  24.0  24.00  24.0
DE                122.0  27.991803  3.016472  23.0  26.0  27.0  30.00  36.0
DT                 98.0  27.918367  2.723224  24.0  26.0  28.0  29.75  35.0
FB                 15.0  27.800000  2.007130  25.0  26.0  28.0  29.50  31.0
FS                 62.0  27.693548  3.065347  23.0  25.0  27.0  29.00  35.0
G                  89.0  27.887640  2.723793  23.0  25.0  28.0  30.00  34.0
ILB                66.0  27.909091  2.364658  24.0  26.0  28.0  30.00  33.0
LB                  1.0  33.000000       NaN  33.0  33.0  33.0  33.00  33.0
MLB                25.0  26.520000  2.329521  23.0  25.0  26.0  27.00  33.0
NT                 29.0  28.793103  3.086599  23.0  27.0  28.0  30.00  37.0
OLB               115.0  27.765217  2.620200  23.0  26.0  27.0  29.50  34.0
QB                 53.0  30.226415  4.956121  24.0  26.0  29.0  34.00  45.0
RB                107.0  27.018692  2.359170  23.0  25.0  27.0  28.00  34.0
SS                 63.0  27.571429  2.692368  23.0  26.0  27.0  29.50  35.0
T                 102.0  28.656863  3.547402  23.0  26.0  28.0  31.00  41.0
TE                106.0  28.198113  2.863298  23.0  26.0  28.0  30.00  38.0
WR                186.0  27.462366  2.797911  23.0  25.0  27.0  29.00  37.0